In [1]:
import cobra
model = cobra.io.read_sbml_model('/data1/xpgeng/cross_pathogen/GEM_model/iML1515.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
import csv
one_gene = {}
with open('/data1/xpgeng/cross_pathogen/FBA/' +'iML1515_1KO_growth', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) 
    for row in reader:
        #print(len(row))
        if len(row) == 4:            
            one_gene[row[0]] = int(row[3])           
    print(len(one_gene)) 


1516


In [3]:
gene_list = []
for k,v in one_gene.items():
    gene_list.append(k)
gene_list.remove('s0001')
gene_list = sorted(gene_list)
print(len(gene_list))

1515


In [ ]:
gene_dic = {}
for i in range(1515):
    gene_dic[gene_list[i]] = i

In [14]:
import csv
from itertools import combinations
from tqdm import tqdm
import cobra.flux_analysis
from multiprocessing import Pool

# Adjust the function to use model and gene_list as arguments
def knockout(gene, gene_list, model):
    
    rest_list = gene_list[gene_dic[gene]+1:]
    twogene = list(combinations(rest_list, 2))
    
    filepath = f'iML1515_{gene}.csv'

    with open('/data1/xpgeng/cross_pathogen/FBA/iML1515-3KO/' + filepath, mode='a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Gene', 'Gene1', 'Gene2', 'Initial Growth', 'Altered Growth', 'Label'])

        initial_state = cobra.flux_analysis.pfba(model)
        initial_growth = round(initial_state.fluxes['BIOMASS_Ec_iML1515_core_75p37M'], 3)

        for gene1, gene2 in tqdm(twogene):
            with model:
                try:
                    model.genes.get_by_id(gene).knock_out()
                    model.genes.get_by_id(gene1).knock_out()
                    model.genes.get_by_id(gene2).knock_out()
                    altered_state = cobra.flux_analysis.pfba(model)
                    altered_growth = round(altered_state.fluxes['BIOMASS_Ec_iML1515_core_75p37M'], 3)
                    label = 1 if altered_growth < 0.1 * initial_growth else 0
                    writer.writerow([gene, gene1, gene2, initial_growth, altered_growth, label])
                except Exception as e:
                    writer.writerow([gene, gene1, gene2, 'Failed', str(e)])

In [18]:
def run_knockouts(gene_list, model):
    # Slice the first 100 genes from the list
    top_genes = gene_list[200:300]
    
    # Create a multiprocessing pool with 100 workers
    with Pool(100) as pool:
        # Each pool worker will execute the `knockout` function with a gene, the full gene list, and the model
        results = pool.starmap(knockout, [(gene, gene_list, model) for gene in top_genes])

if __name__ == "__main__":
    run_knockouts(gene_list, model)

 35%|██████▌            | 258391/742371 [49:08:24<92:02:31,  1.46it/s]





























Process ForkPoolWorker-285:
Process ForkPoolWorker-264:
Process ForkPoolWorker-275:
Process ForkPoolWorker-254:
Process ForkPoolWorker-269:
Process ForkPoolWorker-291:
Process ForkPoolWorker-286:
Process ForkPoolWorker-266:
Process ForkPoolWorker-280:
Process ForkPoolWorker-289:
Process ForkPoolWorker-268:
Process ForkPoolWorker-277:
Process ForkPoolWorker-294:
Process ForkPoolWorker-262:
Process ForkPoolWorker-259:
Process ForkPoolWorker-258:
Process ForkPoolWorker-260:
Process ForkPoolWorker-263:
Process ForkPoolWorker-284:
Process ForkPoolWorker-287:
Process ForkPoolWorker-257:
Process ForkPoolWorker-297:
Process ForkPoolWorker-274:
Process ForkPoolWorker-282:
Process ForkPoolWorker-270:


KeyboardInterrupt: 

b3041 missing some data, re run it

In [7]:
print([i for i in range(100, 200)])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [4]:
from itertools import combinations
twogene=list(combinations(gene_list,2))
from tqdm import tqdm

In [6]:
with open('iML1515_2KO_growth', mode='a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    firstrow = ['gene, label']
    writer.writerow(firstrow)  

    state_0   = cobra.flux_analysis.pfba(model)      
    growth_0    = round(state_0['BIOMASS_Ec_iML1515_core_75p37M'], 3)

    for i in tqdm(twogene):   
        with model:                
            try:            
                model.genes.get_by_id(i[0]).knock_out()
                model.genes.get_by_id(i[1]).knock_out()
                state_1   = cobra.flux_analysis.pfba(model)
                growth_1    = round(state_1['BIOMASS_Ec_iML1515_core_75p37M'], 3)
                if growth_1 < 0.5*growth_0:
                    
                    writer.writerow([i[0], i[1], growth_0, growth_1, 1]) 
                else:
                    writer.writerow([i[0], i[1], growth_0, growth_1, 0])
            except:
                writer.writerow([i, 'fail'])

100%|████████████████████████████████| 860016/860016 [63:12:54<00:00,  3.78it/s]


In [5]:
two_gene = {}
with open('/data1/xpgeng/cross_pathogen/FBA/' +'iML1515_2KO_growth', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) 
    for row in reader:
        #print(len(row))
        if len(row) == 5:            
            two_gene[row[0], row[1]] = int(row[4])           
    print(len(two_gene)) 

860012


In [6]:
two_list = []
for k,v in two_gene.items():
    if v == 1:
        two_list.append(k[0])
        two_list.append(k[1])
print(two_list)

['b2551', 'b0907', 'b2551', 'b4388', 'b2551', 'b2913', 'b0870', 'b0907', 'b0870', 'b2320', 'b2436', 'b3867', 'b3500', 'b0888', 'b2465', 'b2935', 'b2836', 'b0954', 'b4054', 'b0928', 'b3281', 'b1692', 'b0844', 'b3812', 'b0126', 'b0339', 'b2508', 'b0238', 'b2508', 'b2868', 'b2508', 'b2867', 'b2508', 'b2866', 'b3846', 'b0954', 'b0474', 'b4383', 'b0474', 'b1207', 'b0474', 'b4094', 'b3008', 'b1622', 'b1190', 'b4053', 'b0910', 'b0171', 'b3926', 'b3608', 'b4383', 'b1207', 'b2708', 'b3197', 'b0077', 'b3671', 'b0077', 'b3670', 'b0078', 'b3671', 'b0078', 'b3670', 'b1849', 'b2500', 'b0099', 'b1865', 'b3386', 'b2029', 'b0073', 'b1800', 'b2421', 'b2414', 'b0002', 'b3940', 'b1096', 'b4039', 'b3117', 'b3772', 'b0273', 'b4254', 'b3744', 'b0674', 'b3608', 'b1727', 'b1207', 'b4094', 'b2422', 'b0763', 'b2422', 'b0764', 'b2422', 'b0765', 'b0763', 'b2423', 'b0763', 'b2424', 'b0764', 'b2423', 'b0764', 'b2424', 'b2423', 'b0765', 'b0765', 'b2424', 'b3213', 'b1761', 'b1761', 'b3212', 'b0452', 'b0954', 'b0048', 

In [24]:
print(len(set(two_list)))

108
